In [1]:
from pyod.models.ecod import ECOD  # ECOD detector
from pyod.models.knn import KNN   # kNN detector
from joblib import dump, load
from pyod.utils.data import evaluate_print
from pyod.utils.example import visualize
from loader import Loader
import os

## Data Loading and Pre-Processing

In [2]:
loader = Loader()
if not os.path.exists("data/data.pkl"):
    loader.extract()
    df = loader.create_df_from_xlsx()
    df = loader.process_df(df)
    loader.save_df(df)
else:
    df = loader.load_df()


Processing: 100%|██████████| 12872/12872 [00:08<00:00, 1570.13it/s]


In [3]:
print(df.shape)
print(df.columns)
print(df.info())

(12872, 19)
Index(['کد محور', 'مدت زمان کارکرد (دقیقه)', 'تعداد کل وسیله نقلیه',
       'تعداد وسیله نقلیه کلاس ۱', 'تعداد وسیله نقلیه کلاس ۲',
       'تعداد وسیله نقلیه کلاس ۳', 'تعداد وسیله نقلیه کلاس ۴',
       'تعداد وسیله نقلیه کلاس ۵', 'سرعت متوسط', 'تعداد تخلف سرعت غیر مجاز',
       'تعداد تخلف فاصله غیر مجاز', 'تعداد تخلف سبقت غیر مجاز',
       'تعداد برآورد شده', 'تعداد وسیله نقلیه سبک', 'تعداد وسیله نقلیه سنگین',
       'holiday', 'date', 'day_of_week', 'time_start'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
Int64Index: 12872 entries, 7285 to 5118
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   کد محور                    12872 non-null  int64  
 1   مدت زمان کارکرد (دقیقه)    12872 non-null  int64  
 2   تعداد کل وسیله نقلیه       12872 non-null  int64  
 3   تعداد وسیله نقلیه کلاس ۱   12372 non-null  float64
 4   تعداد وسیله نقلیه کلاس ۲   12372 non-null  

In [ ]:
# sfdk: I recommend to use 'تعداد برآورد شده'
# sfdk: We can merge 'تعداد وسیله نقلیه سنگین','تعداد وسیله نقلیه سبک' data with new numerical classes
df.drop(columns=['تعداد وسیله نقلیه سنگین','تعداد وسیله نقلیه سبک','تعداد برآورد شده']).info()

In [ ]:
df = df.drop(columns=['تعداد وسیله نقلیه سنگین','تعداد وسیله نقلیه سبک','تعداد برآورد شده'])

In [ ]:
df.dropna().info()

In [ ]:
df = df.dropna()

## Some Plots

In [ ]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

In [4]:
df.tail()

,کد محور,مدت زمان کارکرد (دقیقه),تعداد کل وسیله نقلیه,تعداد وسیله نقلیه کلاس ۱,تعداد وسیله نقلیه کلاس ۲,تعداد وسیله نقلیه کلاس ۳,تعداد وسیله نقلیه کلاس ۴,تعداد وسیله نقلیه کلاس ۵,سرعت متوسط,تعداد تخلف سرعت غیر مجاز,تعداد تخلف فاصله غیر مجاز,تعداد تخلف سبقت غیر مجاز,تعداد برآورد شده,تعداد وسیله نقلیه سبک,تعداد وسیله نقلیه سنگین,holiday,date,day_of_week,time_start
5149,113474,1370,25340,23255.0,1017.0,676.0,134.0,258.0,71.0,7535,7337,0,27270.0,NaN,NaN,False,1401-01-31,4,00:00:00
4350,113353,1265,27632,25861.0,843.0,271.0,70.0,587.0,65.0,1138,5804,0,34357.0,NaN,NaN,False,1401-01-31,4,00:00:00
5180,113478,1440,39026,33656.0,2728.0,1227.0,416.0,999.0,73.0,509,5255,0,39026.0,NaN,NaN,False,1401-01-31,4,00:00:00
4376,113391,1440,7352,6803.0,166.0,89.0,135.0,159.0,76.0,0,0,0,7352.0,NaN,NaN,False,1401-01-31,4,00:00:00
5118,113469,1435,27239,25437.0,721.0,308.0,77.0,696.0,71.0,471,1901,0,27335.0,NaN,NaN,False,1401-01-31,4,00:00:00


In [ ]:
df.info()

In [5]:
df[['date']].value_counts()

date      
1401-01-01    122
1401-01-02    122
1401-01-03    122
1401-01-04    122
1401-01-07    122
             ... 
1391-01-16      6
1391-01-01      6
1391-01-15      6
1391-01-14      6
1391-01-02      6
Length: 248, dtype: int64

In [ ]:
df.columns

In [ ]:
per_code_dfs = {}
for code in df['کد محور'].unique():
    per_code_dfs[code] = df[df['کد محور']==code]

In [ ]:
per_code_dfs.keys()

In [ ]:
per_code_dfs[113401].info()

In [ ]:
per_code_dfs[113401]['date'].unique()

## Other :)

In [ ]:
roads = loader.get_roads()
print(list(roads.values())[:5])
print(len(roads))

In [ ]:
clf_name = "KNN"
# clf = ECOD()
clf = KNN()


In [ ]:
# TODO
X_train = []

In [ ]:
clf.fit(X_train)

# get the prediction label and outlier scores of the training data
y_train_pred = clf.labels_  # binary labels (0: inliers, 1: outliers)
y_train_scores = clf.decision_scores_  # raw outlier scores

# get the prediction on the test data
y_test_pred = clf.predict(X_test)  # outlier labels (0 or 1)
y_test_scores = clf.decision_function(X_test)  # outlier scores

# it is possible to get the prediction confidence as well
y_test_pred, y_test_pred_confidence = clf.predict(
    X_test, return_confidence=True
)  # outlier labels (0 or 1) and confidence in the range of [0,1]


In [ ]:
# evaluate and print the results
print("\nOn Training Data:")
evaluate_print(clf_name, y_train, y_train_scores)
print("\nOn Test Data:")
evaluate_print(clf_name, y_test, y_test_scores)


In [ ]:
visualize(
    clf_name,
    X_train,
    y_train,
    X_test,
    y_test,
    y_train_pred,
    y_test_pred,
    show_figure=True,
    save_figure=False,
)


In [ ]:
# save the model
dump(clf, "clf.joblib")
# load the model
clf = load("clf.joblib")
